In [10]:
#Reading data from google drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler

SEED = 2017

In [0]:
data = pd.read_csv("gdrive/My Drive/DeepLearningCodeBook/winequality-red.csv",  sep=';');

In [0]:
y = data['quality']
X = data.drop(['quality'], axis = 1)

In [23]:
X

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size = 0.2 , random_state = SEED)

In [25]:
print('average quality of training set : {:.4f}'.format(y_train.mean()))
X_train.head()

average quality of training set : 5.6231


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1140,7.3,0.40,0.30,1.7,0.080,33.0,79.0,0.99690,3.41,0.65,9.5
920,9.6,0.41,0.37,2.3,0.091,10.0,23.0,0.99786,3.24,0.56,10.5
1198,7.7,0.26,0.26,2.0,0.052,19.0,77.0,0.99510,3.15,0.79,10.9
423,10.5,0.24,0.47,2.1,0.066,6.0,24.0,0.99780,3.15,0.90,11.0
601,13.2,0.46,0.52,2.2,0.071,12.0,35.0,1.00060,3.10,0.56,9.0


In [0]:
scaler = StandardScaler().fit(X_train)
X_train =pd.DataFrame(scaler.transform(X_train))
X_test = pd.DataFrame(scaler.transform(X_test))

In [0]:
# model 

model = Sequential()
model.add(Dense(200, input_dim = X_train.shape[1] , activation="relu"))
model.add(Dense(25, activation="relu"))
model.add(Dense(1, activation="linear"))

opt = Adam()

model.compile(loss = 'mse' , optimizer = opt , metrics = ['accuracy']) 

In [0]:
# study these at
# https://stackoverflow.com/questions/43906048/keras-early-stopping

callbacks = [
             EarlyStopping(monitor='val_acc', patience=200, verbose=2),
             ModelCheckpoint('gdrive/My Drive/DeepLearningCodeBook/multi_layer_best_model.h5', monitor='val_acc', save_best_only=True, verbose=0)
            ]

In [0]:
batch_size = 64
n_epochs = 5000

In [43]:
model.fit(X_train.values , y_train , batch_size=batch_size , epochs = n_epochs , validation_split=0.2,     
             verbose=2,
             validation_data=(X_test.values, y_test),
             callbacks=callbacks)

Train on 1279 samples, validate on 320 samples
Epoch 1/5000
 - 0s - loss: 0.2669 - acc: 0.7099 - val_loss: 0.3607 - val_acc: 0.6406
Epoch 2/5000
 - 0s - loss: 0.2640 - acc: 0.7076 - val_loss: 0.3836 - val_acc: 0.6438
Epoch 3/5000
 - 0s - loss: 0.2691 - acc: 0.6794 - val_loss: 0.3805 - val_acc: 0.6062
Epoch 4/5000
 - 0s - loss: 0.2710 - acc: 0.6959 - val_loss: 0.3857 - val_acc: 0.6094
Epoch 5/5000
 - 0s - loss: 0.2650 - acc: 0.7037 - val_loss: 0.3696 - val_acc: 0.6687
Epoch 6/5000
 - 0s - loss: 0.2635 - acc: 0.6935 - val_loss: 0.3637 - val_acc: 0.6312
Epoch 7/5000
 - 0s - loss: 0.2635 - acc: 0.7005 - val_loss: 0.3566 - val_acc: 0.6469
Epoch 8/5000
 - 0s - loss: 0.2615 - acc: 0.6982 - val_loss: 0.3649 - val_acc: 0.6406
Epoch 9/5000
 - 0s - loss: 0.2628 - acc: 0.7084 - val_loss: 0.3710 - val_acc: 0.6562
Epoch 10/5000
 - 0s - loss: 0.2706 - acc: 0.7037 - val_loss: 0.3914 - val_acc: 0.6031
Epoch 11/5000
 - 0s - loss: 0.2556 - acc: 0.7091 - val_loss: 0.3582 - val_acc: 0.6344
Epoch 12/5000
 -

In [44]:
best_model = model
best_model.load_weights('gdrive/My Drive/DeepLearningCodeBook/multi_layer_best_model.h5')
best_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# Evaluate on test set
score = best_model.evaluate(X_test.values, y_test, verbose=0)
print('Test accuracy: %.2f%%' % (score[1]*100))

Test accuracy: 70.00%
